In [ ]:
#imports

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

import matplotlib.pyplot as plt
import seaborn as sns
import os

#regex
import re

In [ ]:
#create an empty list to collect all reviews
reviews  = []

#create an empty list to collect rating stars
stars = []

#create an empty list to collect date
date = []

#create an empty list to collect country the reviewer is from
country = []

In [ ]:
for i in range(1, 36):
    page = requests.get(f"https://www.airlinequality.com/airline-reviews/british-airways/page/{i}/?sortby=post_date%3ADesc&pagesize=100")

    # Change the parser from "html5" to "html.parser"
    soup = BeautifulSoup(page.content, "html.parser")

    for item in soup.find_all("div", class_="text_content"):
        reviews.append(item.text)

    for item in soup.find_all("div", class_="rating-10"):
        try:
            stars.append(item.span.text)
        except:
            print(f"Error on page {i}")
            stars.append("None")

    #date
    for item in soup.find_all("time"):
        date.append(item.text)

    #country
    for item in soup.find_all("h3"):
        country.append(item.span.next_sibling.text.strip(" ()"))


Error on page 31
Error on page 33
Error on page 33
Error on page 35
Error on page 35


In [ ]:
#check the length of total reviews extracted
len(reviews)

3500

In [ ]:
len(country)

3500

In [ ]:
#check the length
stars = stars[:3500]

In [ ]:
#create a dataframe from these collected lists of data

df = pd.DataFrame({"reviews":reviews,"stars": stars, "date":date, "country": country})

In [ ]:
df.head()

,reviews,stars,date,country
0,"✅ Trip Verified | I had a flight from Miami, F...",\n\t\t\t\t\t\t\t\t\t\t\t\t\t5,23rd September 2023,United States
1,✅ Trip Verified | We started our day with BA ...,1,22nd September 2023,United States
2,✅ Trip Verified | I fly British Airways weekl...,1,22nd September 2023,United Kingdom
3,Not Verified | Everything was ok until our co...,1,20th September 2023,France
4,Not Verified | My initial flight was cancelle...,1,20th September 2023,Germany


In [ ]:
df.shape

(3500, 4)

In [ ]:
#Export the data into a csv format

In [ ]:
import os

cwd = os.getcwd()
df.to_csv(cwd+ "/BA_reviews.csv")

In [ ]:
df.head()

,reviews,stars,date,country
0,"✅ Trip Verified | I had a flight from Miami, F...",\n\t\t\t\t\t\t\t\t\t\t\t\t\t5,23rd September 2023,United States
1,✅ Trip Verified | We started our day with BA ...,1,22nd September 2023,United States
2,✅ Trip Verified | I fly British Airways weekl...,1,22nd September 2023,United Kingdom
3,Not Verified | Everything was ok until our co...,1,20th September 2023,France
4,Not Verified | My initial flight was cancelle...,1,20th September 2023,Germany


In [ ]:
cwd = os.getcwd()

df = pd.read_csv(cwd+"/BA_reviews.csv", index_col=0)

In [ ]:
df.head()

,reviews,stars,date,country
0,"✅ Trip Verified | I had a flight from Miami, F...",\n\t\t\t\t\t\t\t\t\t\t\t\t\t5,23rd September 2023,United States
1,✅ Trip Verified | We started our day with BA ...,1,22nd September 2023,United States
2,✅ Trip Verified | I fly British Airways weekl...,1,22nd September 2023,United Kingdom
3,Not Verified | Everything was ok until our co...,1,20th September 2023,France
4,Not Verified | My initial flight was cancelle...,1,20th September 2023,Germany


We will create a column which mentions if the user is verified or not.

In [ ]:
df['verified'] = df.reviews.str.contains("Trip Verified")

In [ ]:
df['verified']

0        True
1        True
2        True
3       False
4       False
        ...  
3495    False
3496    False
3497    False
3498    False
3499    False
Name: verified, Length: 3500, dtype: bool

In [ ]:
#Cleaning Reviews
##We will extract the column of reviews into a separate dataframe and clean it for semantic analysis

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
import re

# Specify the NLTK data directory
nltk.data.path.append("https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml")

# Ensure you have the "stopwords" dataset downloaded
nltk.download('stop-words', download_dir="https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml")

lemma = WordNetLemmatizer()

reviews_data = df.reviews.str.strip("✅ Trip Verified |")

# Create an empty list to collect cleaned data corpus
corpus = []

# Loop through each review, remove punctuations, lowercase it, lemmatize, and add to the corpus
for rev in reviews_data:
    rev = re.sub('[^a-zA-Z]', ' ', rev)
    rev = rev.lower()
    rev = rev.split()
    rev = [lemma.lemmatize(word) for word in rev if word not in nltk.corpus.stopwords.words("english")]
    rev = " ".join(rev)
    corpus.append(rev)

[nltk_data] Error loading stop-words: Package 'stop-words' not found
[nltk_data]     in index


LookupError: ignored

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords

# Print the stopwords
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
print(nltk.data.path)

['/root/nltk_data', '/usr/nltk_data', '/usr/share/nltk_data', '/usr/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data', 'https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml']


In [ ]:
from nltk.stem import WordNetLemmatizer

# Specify the NLTK data directory
#nltk.data.path.append("https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml")

# Ensure you have the "stopwords" dataset downloaded
nltk.download('wordnet')

lemma = WordNetLemmatizer()

reviews_data = df.reviews.str.strip("✅ Trip Verified |")

# Create an empty list to collect cleaned data corpus
corpus = []

# Loop through each review, remove punctuations, lowercase it, lemmatize, and add to the corpus
for rev in reviews_data:
    rev = re.sub('[^a-zA-Z]', ' ', rev)
    rev = rev.lower()
    rev = rev.split()
    rev = [lemma.lemmatize(word) for word in rev if word not in nltk.corpus.stopwords.words("english")]
    rev = " ".join(rev)
    corpus.append(rev)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# add the corpus to the original dataframe

df['corpus'] = corpus

In [ ]:
df.head()

,reviews,stars,date,country,verified,corpus
0,"✅ Trip Verified | I had a flight from Miami, F...",\n\t\t\t\t\t\t\t\t\t\t\t\t\t5,23rd September 2023,United States,True,flight miami florida dublin ireland via london...
1,✅ Trip Verified | We started our day with BA ...,1,22nd September 2023,United States,True,started day ba prague flight actually left tim...
2,✅ Trip Verified | I fly British Airways weekl...,1,22nd September 2023,United Kingdom,True,fly british airway weekly want route option th...
3,Not Verified | Everything was ok until our co...,1,20th September 2023,France,False,verified everything ok connecting flight londo...
4,Not Verified | My initial flight was cancelle...,1,20th September 2023,Germany,False,verified initial flight cancelled hour prior f...


**Cleaning/Fromatting date**

In [ ]:
df.dtypes

reviews     object
stars       object
date        object
country     object
verified      bool
corpus      object
dtype: object

In [ ]:
# convert the date to datetime format

df.date = pd.to_datetime(df.date)

In [ ]:
df.date.head()

0   2023-09-23
1   2023-09-22
2   2023-09-22
3   2023-09-20
4   2023-09-20
Name: date, dtype: datetime64[ns]

Cleaning ratings with stars

In [ ]:
#check for unique values
df.stars.unique()

array(['\n\t\t\t\t\t\t\t\t\t\t\t\t\t5', '1', '6', '2', '8', '3', '4', '7',
       '10', '9', '5', 'None'], dtype=object)

In [ ]:
# remove the \t and \n from the ratings
df.stars = df.stars.str.strip("\n\t\t\t\t\t\t\t\t\t\t\t\t\t")

In [ ]:
df.stars.value_counts()

1       828
2       403
3       390
8       342
10      298
9       289
7       286
5       248
4       240
6       172
None      4
Name: stars, dtype: int64

There are 4 rows having values "None" in the ratings. We will drop all these 4 rows.

In [ ]:
# drop the rows where the value of ratings is None
df.drop(df[df.stars == "None"].index, axis=0, inplace=True)

In [ ]:
#check the unique values again
df.stars.unique()

array(['5', '1', '6', '2', '8', '3', '4', '7', '10', '9'], dtype=object)

Check for null Values

In [ ]:
df.isnull().value_counts()

reviews  stars  date   country  verified  corpus
False    False  False  False    False     False     3494
                       True     False     False        2
dtype: int64

In [ ]:
df.country.isnull().value_counts()

False    3494
True        2
Name: country, dtype: int64

We have two missing values for country. For this we can just remove those two reviews (rows) from the dataframe.

In [ ]:
#drop the rows using index where the country value is null
df.drop(df[df.country.isnull() == True].index, axis=0, inplace=True)

In [ ]:
df.shape

(3494, 6)

In [ ]:
#resetting the index
df.reset_index(drop=True)

,reviews,stars,date,country,verified,corpus
0,"✅ Trip Verified | I had a flight from Miami, F...",5,2023-09-23,United States,True,flight miami florida dublin ireland via london...
1,✅ Trip Verified | We started our day with BA ...,1,2023-09-22,United States,True,started day ba prague flight actually left tim...
2,✅ Trip Verified | I fly British Airways weekl...,1,2023-09-22,United Kingdom,True,fly british airway weekly want route option th...
3,Not Verified | Everything was ok until our co...,1,2023-09-20,France,False,verified everything ok connecting flight londo...
4,Not Verified | My initial flight was cancelle...,1,2023-09-20,Germany,False,verified initial flight cancelled hour prior f...
...,...,...,...,...,...,...
3489,Flew JNB-LHR return. Only considered BA becaus...,1,2014-07-15,South Africa,False,flew jnb lhr return considered ba fly route un...
3490,Very disappointed with my flight back from Mun...,10,2014-07-15,United Kingdom,False,disappointed flight back munich although hour ...
3491,I flew with British Airways with my mother fro...,8,2014-07-15,Italy,False,flew british airway mother paphos manchester f...
3492,Flew LHR to Mumbai return with my husband - ve...,4,2014-07-15,United Kingdom,False,flew lhr mumbai return husband disappointing g...


Now our data is all cleaned and ready for data visualization and data analysis.

In [ ]:
# export the cleaned data

df.to_csv("/cleaned-BA-reviews.csv")

In [ ]:
/cleaned-BA-reviews.csv